In [1]:
import d00_utils.read_log as dr
# read log file to lines and
# read log file
log_variable_folder_path = os.path.join(
    os.path.expanduser("~"),
    'simulation',
    'lammps_process',
    'data',
    '01_raw',
    'Link to lmp_run',
    'test',
    '20200511',
    'f_5e6',
)


In [2]:
log_variable = dr.get_log_variable_dic_fromlog_variable(log_variable_folder_path)


In [3]:
import pickle
import json
folder_path_list_last_to_initial = [log_variable_folder_path]
log_variable = dr.get_log_variable_dic_fromlog_variable(log_variable_folder_path)
log_variable_dic_list_last_to_initial = [log_variable]
parent_folder_path = os.path.abspath(os.path.join(log_variable_folder_path, os.pardir))
while log_variable['rst_from'] != '0' and os.path.isfile(os.path.join(parent_folder_path, 'log.lammps')):
    log_variable = dr.get_log_variable_dic_fromlog_variable(parent_folder_path)
    log_variable_dic_list_last_to_initial.append(
        log_variable
    )
    folder_path_list_last_to_initial.append(
        parent_folder_path
    )
log_variable_dic_list = log_variable_dic_list_last_to_initial[::-1]
if log_variable_dic_list[0]['rst_from'] != '0':
    sys.exit('log_variable wrong')

n_simu_total = len(log_variable_dic_list)
folder_path_list_initial_to_last = folder_path_list_last_to_initial[::-1]

# save all global variable get from log_variable to pickle
with open(log_variable_folder_path + 'python_global.pckl', 'wb') as f:
    pickle.dump([log_variable_dic_list, n_simu_total, log_variable, folder_path_list_initial_to_last], f)
with open(log_variable_folder_path + 'python_global.json', 'w') as f:
    json.dump([log_variable_dic_list, n_simu_total, log_variable, folder_path_list_initial_to_last], f)

In [4]:
len(log_variable_dic_list)

2

In [2]:
# get input parameters from lines in log files
# create log_variable dictionary from log file
# write values of variables to dictionary 
log_variable_name = 'log.lammps'
log_file_path = os.path.join(log_variable_folder_path, log_variable_name)
if os.path.isfile(log_file_path):
    with open(log_file_path, mode='r') as f:
        lines = f.read().strip().split('\n')
else:
    sys.exit("log file not exist")

lines_start_variable = [line for line in lines if line.startswith("variable")]
variable_names = [line.split()[1] for line in lines_start_variable]
log_variable = dict.fromkeys(variable_names)
# get rst_from
rst_from = int(
    [line for line in lines_start_variable if line.split()[1] == 'rst_from'][0].split()[3]
)

# get variable
for variable_name in log_variable.keys():
    satisfy_lines = [line for line in lines_start_variable if line.split()[1] == variable_name]
    
    if len(satisfy_lines) != 0:
        first_line_words = satisfy_lines[0].split()
    
        if first_line_words[2] == "index" and first_line_words[3][0] != '$':
            variable_value = first_line_words[3]

        elif first_line_words[2] == "index" and first_line_words[3][0] == '$':
            second_satisfy_line = satisfy_lines[1]
            second_line_words = second_satisfy_line.split()
            variable_value = second_line_words[3]

        elif first_line_words[2] == "equal" or first_line_words[2] == "string":
            last_satisfy_line = satisfy_lines[-1]
            last_line_words = last_satisfy_line.split()
            variable_value = last_line_words[3]
        
        elif first_line_words[2] == "getenv":
            variable_value = first_line_words[3]

        else:
            pass
        log_variable[variable_name] = variable_value

    else:
        sys.exit("can not find variable {} in log file".format(variable_name))
# write shearwall parameters to log_variable dictionary
if "if_inwall_wall_gran" in log_variable.keys():
    if log_variable["if_inwall_wall_gran"]=="yes":
        log_variable["shearwall"] = "yplane"
    else:
        for line in lines:
            if line.startswith("fix") and line.split()[3] == "wall/gran":
                if line.split()[1] == "inwall": 
                    log_variable["shearwall"] = line.split()[11]
                    break
                elif line.split()[1] == "y_bottom":
                    log_variable["shearwall"] = line.split()[11]
                    break
                else:
                    sys.exit("shearwall missed")
else:
    for line in lines:
        if line.startswith("fix") and line.split()[3] == "wall/gran":
            if line.split()[1] == "inwall": 
                log_variable["shearwall"] = line.split()[11]
                break
            elif line.split()[1] == "y_bottom":
                log_variable["shearwall"] = line.split()[11]
                break
            else:
                sys.exit("shearwall missed")

# write chunk_2_3 parameters to log_variable dictionary
# select compute line
lines_start_compute = [line for line in lines if line.startswith("compute")]
# get chunk 23 info
satisfy_lines = [line for line in lines_start_compute if line.split()[3] == 'chunk/atom' and line.split()[1] == "chunk_2_3"]
# get chunk/atom
if len(satisfy_lines) != 0:
    log_variable["chunk/atom 23"] = [
        satisfy_lines[0].split()[1],
        satisfy_lines[0].split()[5],
        satisfy_lines[0].split()[4],
    ]
else:
    pass